<a href="https://colab.research.google.com/github/raz0208/Techniques-For-Text-Analysis/blob/main/SentimentClassificationByLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sentiment classification
Sentiment classification is a type of natural language processing (NLP) task that involves determining the emotional tone or opinion expressed in a piece of text. In practice, this means categorizing text—like reviews, social media posts, or comments—as having positive, negative, neutral, or even more fine-grained sentiments.

### Key Points:
### Definition:
- It’s the process of assigning sentiment labels (such as positive, negative, or neutral) to a given text.

### Techniques:

- Lexicon-based Methods: Use predefined dictionaries of words that are associated with specific sentiments.
- Machine Learning Approaches: Train models like Naive Bayes, Support Vector Machines (SVM), or logistic regression on labeled datasets to predict sentiment.
- Deep Learning Methods: Leverage neural networks, such as LSTMs or Transformers, which can capture context and nuances in language for improved performance.

### Applications:

- Customer Feedback: Automatically analyzing reviews and feedback to gauge customer satisfaction.
- Social Media Monitoring: Tracking public sentiment towards brands, events, or political issues.
- Market Research: Helping companies understand public opinion trends and make data-driven decisions.

Overall, sentiment classification helps in summarizing and quantifying subjective information, making it easier for organizations to understand and react to the emotions behind the text data.

### Step 1: Import libraries and read dataset

In [34]:
# Import rewuired libraries
import torch
import numpy as np
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from torch.utils.data import DataLoader, TensorDataset

In [35]:
# Load Data (Example dataset)
data = [
    ("I love this product!", 1),
    ("This is the worst experience I've had.", 0),
    ("Not great, but not terrible either.", 0),
    ("Absolutely fantastic!", 1),
    ("I wouldn't recommend this to anyone.", 0),
    ("A good choice to buy.", 1),
    ("It is not great but it has some use.", 0),
    ("I did not like this product.", 0),
    ("Not good.", 0),
    ("It was very useful.", 1),
    ("I surperised of this product.", 1),
    ("An amazing shooping.", 1),
    ("I am disappointed, not recommend to anyone.", 0),
    ("You will regret to spend mony on this product.", 0),
    ("Buy and enjoy, It is great.", 0)

]
texts, labels = zip(*data)
texts, labels

(('I love this product!',
  "This is the worst experience I've had.",
  'Not great, but not terrible either.',
  'Absolutely fantastic!',
  "I wouldn't recommend this to anyone.",
  'A good choice to buy.',
  'It is not great but it has some use.',
  'I did not like this product.',
  'Not good.',
  'It was very useful.',
  'I surperised of this product.',
  'An amazing shooping.',
  'I am disappointed, not recommend to anyone.',
  'You will regret to spend mony on this product.',
  'Buy and enjoy, It is great.'),
 (1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0))

### Step 2: Split data into train and test
- Split into train (80%) and test (20%) sets.

In [36]:
# Split data into train and test
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

### Step 3: Load Pretrained Model & Tokenizer
- Use DistilBertTokenizer and DistilBertForSequenceClassification from Hugging Face.

In [37]:
# Load Pretrained Model & Tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Step 4: Tokenization
- Convert texts into tokenized format using DistilBERT tokenizer.
- Apply truncation & padding (max length = 512).

In [38]:
# Tokenization
def tokenize_texts(texts):
    return tokenizer(list(texts), padding=True, truncation=True, max_length=512, return_tensors="pt")

tokenized_train = tokenize_texts(train_texts)
tokenized_test = tokenize_texts(test_texts)

print(tokenized_train, "\n")
print(tokenized_test)

{'input_ids': tensor([[  101,  2017,  2097,  9038,  2000,  5247, 12256,  2100,  2006,  2023,
          4031,  1012,   102],
        [  101,  1037,  2204,  3601,  2000,  4965,  1012,   102,     0,     0,
             0,     0,     0],
        [  101,  2025,  2204,  1012,   102,     0,     0,     0,     0,     0,
             0,     0,     0],
        [  101,  2025,  2307,  1010,  2021,  2025,  6659,  2593,  1012,   102,
             0,     0,     0],
        [  101,  2023,  2003,  1996,  5409,  3325,  1045,  1005,  2310,  2018,
          1012,   102,     0],
        [  101,  4965,  1998,  5959,  1010,  2009,  2003,  2307,  1012,   102,
             0,     0,     0],
        [  101,  1045,  2876,  1005,  1056, 16755,  2023,  2000,  3087,  1012,
           102,     0,     0],
        [  101,  1045,  2106,  2025,  2066,  2023,  4031,  1012,   102,     0,
             0,     0,     0],
        [  101,  1045,  7505,  4842,  5084,  1997,  2023,  4031,  1012,   102,
             0,     0,     

### Step 5: Convert to Tensors

In [39]:
# Convert to Tensors
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

train_dataset = TensorDataset(tokenized_train["input_ids"], tokenized_train["attention_mask"], train_labels)
test_dataset = TensorDataset(tokenized_test["input_ids"], tokenized_test["attention_mask"], test_labels)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

### Step 6: Feature Extraction with DistilBERT
- Pass input through DistilBERT model.
- Extract CLS token embeddings as feature vectors.

In [40]:
# Feature Extraction with DistilBERT
def extract_features(dataloader, model):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, batch_labels = batch
            outputs = model.distilbert(input_ids, attention_mask=attention_mask)
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # CLS token embedding
            features.extend(cls_embeddings)
            labels.extend(batch_labels.cpu().numpy())
    return np.array(features), np.array(labels)

train_features, train_labels = extract_features(train_loader, model)
test_features, test_labels = extract_features(test_loader, model)

print(train_features, train_labels, "\n")
print(test_features, test_labels)

[[-0.02096417  0.03251809 -0.05091907 ... -0.06217223  0.29572147
   0.2798872 ]
 [-0.0323972  -0.13464235  0.05600379 ... -0.03809529  0.2934187
   0.18036121]
 [ 0.10513807 -0.11605074  0.22850509 ... -0.03897438  0.34371826
   0.18249625]
 ...
 [-0.13017929 -0.02802864  0.23941529 ... -0.16929476  0.25457275
   0.26665863]
 [ 0.13890916  0.03303748 -0.00232603 ...  0.06351364  0.26527876
   0.2247639 ]
 [ 0.05618943  0.08847015  0.02758759 ... -0.06055192  0.21250577
   0.23638676]] [0 0 0 0 0 1 1 0 0 1 0 0] 

[[-1.4675239e-01 -2.6985919e-04  2.9537434e-02 ... -9.5808700e-02
   7.6908693e-02  2.9369989e-01]
 [-1.4042497e-01 -1.9086243e-01  2.2226458e-02 ... -1.4932004e-01
   2.2279519e-01  1.7671044e-01]
 [ 2.9251484e-02  8.1084915e-02  8.2918651e-02 ... -1.4410514e-01
   2.8627998e-01  1.8182299e-01]] [1 1 1]


### STep 7: Hyperparameter Tuning

In [41]:
# Hyperparameter Tuning (Example: setting C value for Logistic Regression)
C_values = [0.01, 0.1, 1, 10]
best_C = 1  # Placeholder, can be selected via GridSearchCV

### Step 8: Train Logistic Regression

In [42]:
# Train Logistic Regression
clf = LogisticRegression(C=best_C, max_iter=1000)
clf.fit(train_features, train_labels)

LogisticRegression(C=1, max_iter=1000)

### Step 9: Evaluate Model

In [43]:
# Evaluate Model
y_pred = clf.predict(test_features)
accuracy = accuracy_score(test_labels, y_pred)
report = classification_report(test_labels, y_pred)

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [44]:
# Visualise
print(f"Accuracy: {accuracy:.4f}")
print(report)

Accuracy: 0.6667
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.67      0.80         3

    accuracy                           0.67         3
   macro avg       0.50      0.33      0.40         3
weighted avg       1.00      0.67      0.80         3

